In [2]:
import torch
from torch import nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# .to(device)
# !nvidia-smi\

Using device: cuda:0


In [4]:
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision

In [4]:
model = nn.Sequential(
            nn.Linear(in_ch, 32),
            nn.ReLU(),
            nn.Linear(32, out_ch, bias=False),
            nn.ReLU()
            
        )

class SimpleModel(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_ch, 32),
            nn.ReLU(),
            nn.Linear(32, out_ch, bias=False),
            nn.ReLU()
            
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_ch, 32),
            nn.ReLU(),
            nn.Linear(32, out_ch, bias=False),
            nn.ReLU()
            
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.decoder(x)

model = SimpleModel(64, 10).to(device)

x = torch.rand(4, 64).to(device)
y = torch.rand(4, 10).to(device)

y_pred = model(x)
y_pred

    

NameError: name 'in_ch' is not defined

In [19]:
class SecondModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),          
            nn.Linear(64, 16),
            nn.Tanh(),        
            nn.Linear(16, 4),   
            nn.Softmax(dim=1)    
        )
        
    def forward(self, x):
        return self.model(x)


model = SecondModel().to(device)
x = torch.randn(8,256).to(device)
y = model(x)

print(f'Размерность выходного массива: {y.shape}')
print(f'Сам массив: {y}')

Размерность выходного массива: torch.Size([8, 4])
Сам массив: tensor([[0.2189, 0.2574, 0.2755, 0.2481],
        [0.2125, 0.2939, 0.2289, 0.2647],
        [0.2407, 0.2486, 0.2497, 0.2609],
        [0.2639, 0.2832, 0.2375, 0.2154],
        [0.2011, 0.2551, 0.2959, 0.2478],
        [0.2624, 0.2473, 0.2456, 0.2447],
        [0.2320, 0.2279, 0.2750, 0.2651],
        [0.2198, 0.2167, 0.2457, 0.3178]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


In [39]:
"""19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16"""

class ThirdModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.model(x)


model = ThirdModel().to(device)

x = torch.randn(5, 3, 19, 19).to(device)
y = model(x)

print(f'Размерность выходного массива: {y.shape}')
# print(f'Сам массив: {y}')


Размерность выходного массива: torch.Size([5, 16, 4, 4])


In [37]:
class CombinedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fst = SimpleModel(in_ch=19*19, out_ch=256)
        self.snd = SecondModel()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fst(x)
        x = self.snd(x)
        return x

model = CombinedModel().to(device)

x = torch.randn(5,1,19,19).to(device)

y = model(x)

print(f'Размерность выходного массива: {y.shape}')
print(f'Сам массив: {y}')

Размерность выходного массива: torch.Size([5, 4])
Сам массив: tensor([[0.2836, 0.2546, 0.2176, 0.2442],
        [0.2863, 0.2532, 0.2197, 0.2407],
        [0.2862, 0.2535, 0.2222, 0.2381],
        [0.2784, 0.2632, 0.2166, 0.2418],
        [0.2831, 0.2553, 0.2295, 0.2321]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


In [8]:
#13 - LAB
transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.5), (0.5))
                                            ])

# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./MNIST/train", train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)

test_dataset = torchvision.datasets.MNIST(
    root="./MNIST/test", train=False,
    transform=torchvision.transforms.ToTensor(),
    download=False)
transform = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.5), (0.5))
                                            ])

# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./MNIST/train", train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)

test_dataset = torchvision.datasets.MNIST(
    root="./MNIST/test", train=False,
    transform=torchvision.transforms.ToTensor(),
    download=False)

def encode_label(j):
    # 5 -> [[0], [0], [0], [0], [0], [1], [0], [0], [0], [0]]
    e = np.zeros((10,1))
    e[j] = 1.0
    return e

def shape_data(data):
    features = [np.reshape(x[0][0].numpy(), (784,1)) for x in data]
    labels = [encode_label(y[1]) for y in data]
    return zip(features, labels)

test1 = train_dataset[0]
test2 = [test1]
reshape = shape_data(test2)
list(reshape)
train = shape_data(train_dataset)
test = shape_data(test_dataset)
train = list(train)
test = list(test)
print("datasets is UpToDate")

datasets is UpToDate


In [10]:
class MNISTO(nn.Module):
    def __init__(self):
        super().__init__()
        self.fst_layer = nn.Linear(28 * 28, 128)
        self.snd_layer = nn.Linear(128,64)
        self.trd_layer = nn.Linear(64,10)

    def forward(self, x):
        # Преобразование входных данных в вектор (batch_size, 28*28)
        x = x.view(-1, 28 * 28)
        # Первый слой + ReLU
        x = torch.relu(self.fst_layer(x))
        # Второй слой + ReLU
        x = torch.relu(self.snd_layer(x))
        # Выходной слой + Softmax
        x = torch.softmax(self.trd_layer(x), dim=1)
        return x

model = MNISTO().to(device)



def prepare_data(data):
    features = [torch.tensor(feature, dtype=torch.float32).T.to(device) for feature, _ in data]
    labels = [torch.tensor(label, dtype=torch.float32).squeeze().argmax().to(device) for _, label in data]
    return list(zip(features, labels))

train_data = prepare_data(train)
test_data = prepare_data(test)

print("Data prepared")
    

Data prepared


In [15]:
learning_rate = 0.001
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()  # Устанавливаем модель в режим обучения
    total_loss = 0
    for feature, label in train_data[:5000]:
        # Прямой проход
        output = model(feature.unsqueeze(0))  # Добавляем размер батча
        loss = loss_fn(output, label.unsqueeze(0))  # Loss ожидает батч, поэтому добавляем размерность

        # Обратный проход и обновление весов
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

# Оценка модели
model.eval()  # Устанавливаем модель в режим оценки
correct = 0
total = len(test_data)
with torch.no_grad():
    for feature, label in test_data:
        output = model(feature.unsqueeze(0))
        predicted = torch.argmax(output, dim=1)
        correct += (predicted == label).item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.2%}")

Epoch 1/100, Loss: 7511.6784
Epoch 2/100, Loss: 7512.4718
Epoch 3/100, Loss: 7575.7187
Epoch 4/100, Loss: 7636.5665
Epoch 5/100, Loss: 7542.9576
Epoch 6/100, Loss: 7533.8996
Epoch 7/100, Loss: 7529.8857
Epoch 8/100, Loss: 7528.4128
Epoch 9/100, Loss: 7509.0405
Epoch 10/100, Loss: 7537.9921
Epoch 11/100, Loss: 7506.5427
Epoch 12/100, Loss: 7537.8840
Epoch 13/100, Loss: 7608.2051
Epoch 14/100, Loss: 7506.7907
Epoch 15/100, Loss: 7526.9708
Epoch 16/100, Loss: 7549.7040
Epoch 17/100, Loss: 7521.5237
Epoch 18/100, Loss: 7518.5095
Epoch 19/100, Loss: 7526.9869
Epoch 20/100, Loss: 7544.4642
Epoch 21/100, Loss: 7612.8335
Epoch 22/100, Loss: 7503.3767
Epoch 23/100, Loss: 7483.1296
Epoch 24/100, Loss: 7512.3214
Epoch 25/100, Loss: 7539.5740
Epoch 26/100, Loss: 7491.2384
Epoch 27/100, Loss: 7499.7509
Epoch 28/100, Loss: 7490.7639
Epoch 29/100, Loss: 7507.4950
Epoch 30/100, Loss: 7504.3880
Epoch 31/100, Loss: 7474.5914
Epoch 32/100, Loss: 7508.2454
Epoch 33/100, Loss: 7513.4105
Epoch 34/100, Loss: